In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_log_error
import itertools
import math
import re
import lightgbm as lgb

In [2]:
idx = pd.IndexSlice

In [3]:
train =pd.read_csv("data/train.csv")
test=pd.read_csv("data/test.csv")
oil=pd.read_csv("data/oil.csv")
stores=pd.read_csv("data/stores.csv")
transactions=pd.read_csv("data/transactions.csv")
holidays=pd.read_csv("data/holidays_events.csv")

In [4]:
train['test'] = 0
test['test'] = 1

data = pd.concat([train, test], axis=0)

In [5]:
data = data.merge(holidays, on='date', how='left')
data= data.merge(stores, on='store_nbr', how='left')
data= data.merge(oil, on='date', how='left')
data= data.merge(transactions, on=['date', 'store_nbr'], how='left')
data = data.set_index(['store_nbr', 'date', 'family'])
data = data.drop(index='2013-01-01', level=1)
data

id   sales  onpromotion  \
store_nbr date       family                                                     
1         2013-01-02 AUTOMOTIVE                     1782     2.0            0   
                     BABY CARE                      1783     0.0            0   
                     BEAUTY                         1784     2.0            0   
                     BEVERAGES                      1785  1091.0            0   
                     BOOKS                          1786     0.0            0   
...                                                  ...     ...          ...   
9         2017-08-31 POULTRY                     3029395     NaN            1   
                     PREPARED FOODS              3029396     NaN            0   
                     PRODUCE                     3029397     NaN            1   
                     SCHOOL AND OFFICE SUPPLIES  3029398     NaN            9   
                     SEAFOOD                     3029399     NaN            0   

                                                 test type_x locale  \
store_nbr date       family                                           
1         2013-01-02 AUTOMOTIVE                     0    NaN    NaN   
                     BABY CARE                      0    NaN    NaN   
                     BEAUTY                         0    NaN    NaN   
                     BEVERAGES                      0    NaN    NaN   
                     BOOKS                          0    NaN    NaN   
...                                               ...    ...    ...   
9         2017-08-31 POULTRY                        1    NaN    NaN   
                     PREPARED FOODS                 1    NaN    NaN   
                     PRODUCE                        1    NaN    NaN   
                     SCHOOL AND OFFICE SUPPLIES     1    NaN    NaN   
                     SEAFOOD                        1    NaN    NaN   

                                                locale_name description  \
store_nbr date       family                                               
1         2013-01-02 AUTOMOTIVE                         NaN         NaN   
                     BABY CARE                          NaN         NaN   
                     BEAUTY                             NaN         NaN   
                     BEVERAGES                          NaN         NaN   
                     BOOKS                              NaN         NaN   
...                                                     ...         ...   
9         2017-08-31 POULTRY                            NaN         NaN   
                     PREPARED FOODS                     NaN         NaN   
                     PRODUCE                            NaN         NaN   
                     SCHOOL AND OFFICE SUPPLIES         NaN         NaN   
                     SEAFOOD                            NaN         NaN   

                                                transferred   city      state  \
store_nbr date       family                                                     
1         2013-01-02 AUTOMOTIVE                         NaN  Quito  Pichincha   
                     BABY CARE                          NaN  Quito  Pichincha   
                     BEAUTY                             NaN  Quito  Pichincha   
                     BEVERAGES                          NaN  Quito  Pichincha   
                     BOOKS                              NaN  Quito  Pichincha   
...                                                     ...    ...        ...   
9         2017-08-31 POULTRY                            NaN  Quito  Pichincha   
                     PREPARED FOODS                     NaN  Quito  Pichincha   
                     PRODUCE                            NaN  Quito  Pichincha   
                     SCHOOL AND OFFICE SUPPLIES         NaN  Quito  Pichincha   
                     SEAFOOD                            NaN  Quito  Pichincha   

                                              

In [6]:
data_ = data.copy().reset_index()

train = data_[data_['test'] == 0]
test = data_[data_['test'] == 1]

train['date'] = pd.to_datetime(train["date"])
train['day_of_week'] = train['date'].dt.day_of_week
train['day_of_year'] = train['date'].dt.dayofyear
train['day_of_month'] = train['date'].dt.day
train['month'] = train['date'].dt.month
train['quarter'] = train['date'].dt.quarter
train['year'] = train['date'].dt.year

C:\Users\ivan.trujillo_blueta\AppData\Local\Temp\ipykernel_4804\1357458643.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['date'] = pd.to_datetime(train["date"])
C:\Users\ivan.trujillo_blueta\AppData\Local\Temp\ipykernel_4804\1357458643.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['day_of_week'] = train['date'].dt.day_of_week
C:\Users\ivan.trujillo_blueta\AppData\Local\Temp\ipykernel_4804\1357458643.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [7]:
grouping_columns = ['year', 'quarter', 'month', 'day_of_week', 'day_of_year', 'day_of_month']

for ind, column in enumerate(grouping_columns):
    grouped_data = train.groupby(column)['sales'].sum()
    grouped_data = pd.DataFrame(grouped_data).reset_index()

In [8]:
data_ = data.copy().reset_index()
grouped_data = data_.groupby(['store_nbr', 'family'])

In [9]:
data_

,store_nbr,date,family,id,sales,onpromotion,test,type_x,locale,locale_name,description,transferred,city,state,type_y,cluster,dcoilwtico,transactions
0,1,2013-01-02,AUTOMOTIVE,1782,2.0,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,93.14,2111.0
1,1,2013-01-02,BABY CARE,1783,0.0,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,93.14,2111.0
2,1,2013-01-02,BEAUTY,1784,2.0,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,93.14,2111.0
3,1,2013-01-02,BEVERAGES,1785,1091.0,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,93.14,2111.0
4,1,2013-01-02,BOOKS,1786,0.0,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,93.14,2111.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3081073,9,2017-08-31,POULTRY,3029395,NaN,1,1,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,B,6,47.26,NaN
3081074,9,2017-08-31,PREPARED FOODS,3029396,NaN,0,1,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,B,6,47.26,NaN
3081075,9,2017-08-31,PRODUCE,3029397,NaN,1,1,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,B,6,47.26,NaN
3081076,9,2017-08-31,SCHOOL AND OFFICE SUPPLIES,3029398,NaN,9,1,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,B,6,47.26,NaN


In [11]:
grouped_data.head()

,store_nbr,date,family,id,sales,onpromotion,test,type_x,locale,locale_name,description,transferred,city,state,type_y,cluster,dcoilwtico,transactions
0,1,2013-01-02,AUTOMOTIVE,1782,2.000,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,93.14,2111.0
1,1,2013-01-02,BABY CARE,1783,0.000,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,93.14,2111.0
2,1,2013-01-02,BEAUTY,1784,2.000,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,93.14,2111.0
3,1,2013-01-02,BEVERAGES,1785,1091.000,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,93.14,2111.0
4,1,2013-01-02,BOOKS,1786,0.000,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,93.14,2111.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8905,9,2013-01-06,POULTRY,10687,604.244,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,B,6,NaN,2635.0
8906,9,2013-01-06,PREPARED FOODS,10688,82.000,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,B,6,NaN,2635.0
8907,9,2013-01-06,PRODUCE,10689,0.000,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,B,6,NaN,2635.0
8908,9,2013-01-06,SCHOOL AND OFFICE SUPPLIES,10690,0.000,0,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,B,6,NaN,2635.0


In [12]:
alphas = [0.95, 0.8, 0.65, 0.5]
lags =[1,7,30]

for a in alphas:
    for i in lags:
        data_[f'sales_lag_{i}_alpha_{a}'] = np.log1p(grouped_data['sales'].transform(lambda x: x.shift(i).ewm(alpha=a, min_periods=1).mean()))

data_['sales_lag_7_alpha_0.5'].describe()

count    3.068604e+06
mean     3.002727e+00
std      2.665066e+00
min      0.000000e+00
25%      1.550843e-02
50%      2.562952e+00
75%      5.312741e+00
max      1.112374e+01
Name: sales_lag_7_alpha_0.5, dtype: float64

In [13]:
data_['date'] = pd.to_datetime(data_["date"])
data_['day_of_week'] = data_['date'].dt.day_of_week
data_['day_of_year'] = data_['date'].dt.dayofyear
data_['day_of_month'] = data_['date'].dt.day
data_['month'] = data_['date'].dt.month
data_['quarter'] = data_['date'].dt.quarter
data_['year'] = data_['date'].dt.year

In [14]:
data_['onpromotion'] = data_['onpromotion'].apply(lambda x: x > 0)

In [15]:
sales_lag_columns = list(data_.filter(like="lag").columns)

In [16]:
training_percentage = 0.8
testing_percentage = 0.2

to_dummies = ['day_of_week', 'day_of_month', 'month', 'quarter', 'year', 'store_nbr', 'type_y', 'cluster', 'family', 'onpromotion', 'type_x',
       'locale', 'locale_name', 'city', 'state']

X = data_.loc[:, [ 'day_of_week', 'day_of_month', 'month', 'quarter', 'year', 'store_nbr', 'type_y', 'cluster', 'family', 'onpromotion', 'type_x',
       'locale', 'locale_name',  'city', 'state', 'test', 'sales', 'id']+ sales_lag_columns]
X[to_dummies] = X[to_dummies].astype('category')

data_train = X[X['test'] == 0]
data_test = X[X['test'] == 1]

n = len(data_train)

training_start = 0
training_end = math.floor(n * training_percentage)
validation_start = training_end
validation_end = n

X_train = data_train.loc[training_start:training_end, :].drop(['test', 'sales', 'id'],  axis=1)
y_train = data_train.loc[training_start:training_end, 'sales']
X_val = data_train.loc[validation_start:validation_end,  :].drop(['test', 'sales', 'id'],  axis=1)
y_val = data_train.loc[validation_start:validation_end, 'sales']

X_test = data_test.loc[:, ].drop(['test', 'sales', 'id'],  axis=1)

In [17]:
X_train

,day_of_week,day_of_month,month,quarter,year,store_nbr,type_y,cluster,family,onpromotion,...,sales_lag_30_alpha_0.95,sales_lag_1_alpha_0.8,sales_lag_7_alpha_0.8,sales_lag_30_alpha_0.8,sales_lag_1_alpha_0.65,sales_lag_7_alpha_0.65,sales_lag_30_alpha_0.65,sales_lag_1_alpha_0.5,sales_lag_7_alpha_0.5,sales_lag_30_alpha_0.5
0,2,2,1,1,2013,1,D,13,AUTOMOTIVE,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,1,1,2013,1,D,13,BABY CARE,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,1,1,2013,1,D,13,BEAUTY,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,2,1,1,2013,1,D,13,BEVERAGES,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,2,1,1,2013,1,D,13,BOOKS,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2442048,0,12,9,3,2016,29,E,10,HOME AND KITCHEN I,False,...,3.517793,3.287449e+00,3.554960e+00,3.477677,3.244715e+00,3.618036,3.422472,3.224392,3.633657,3.376880
2442049,0,12,9,3,2016,29,E,10,HOME AND KITCHEN II,False,...,3.132426,3.370022e+00,3.100416e+00,3.111519,3.269981e+00,3.162483,3.071158,3.169575,3.179351,3.011050
2442050,0,12,9,3,2016,29,E,10,HOME APPLIANCES,False,...,0.048671,4.699072e-14,7.342300e-10,0.175633,1.941933e-09,0.000001,0.267830,0.000002,0.000110,0.318454
2442051,0,12,9,3,2016,29,E,10,HOME CARE,True,...,5.632373,5.666043e+00,5.837101e+00,5.626699,5.591884e+00,5.889931,5.610292,5.518580,5.910733,5.590970


In [18]:
X_test

,day_of_week,day_of_month,month,quarter,year,store_nbr,type_y,cluster,family,onpromotion,...,sales_lag_30_alpha_0.95,sales_lag_1_alpha_0.8,sales_lag_7_alpha_0.8,sales_lag_30_alpha_0.8,sales_lag_1_alpha_0.65,sales_lag_7_alpha_0.65,sales_lag_30_alpha_0.65,sales_lag_1_alpha_0.5,sales_lag_7_alpha_0.5,sales_lag_30_alpha_0.5
3052566,2,16,8,3,2017,1,D,13,AUTOMOTIVE,False,...,1.101867,1.489392e+00,2.016376e+00,1.147818,1.417390e+00,1.983362,1.243070,1.390546,1.958797,1.378887
3052567,2,16,8,3,2017,1,D,13,BABY CARE,False,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
3052568,2,16,8,3,2017,1,D,13,BEAUTY,True,...,1.767662,1.651592e+00,1.547290e+00,1.704973,1.644551e+00,1.519433,1.655978,1.620356,1.506958,1.623241
3052569,2,16,8,3,2017,1,D,13,BEVERAGES,True,...,7.749193,7.573078e+00,7.746335e+00,7.681545,7.547668e+00,7.731619,7.636089,7.511847,7.706068,7.616486
3052570,2,16,8,3,2017,1,D,13,BOOKS,False,...,0.667951,2.621440e-11,4.095999e-07,0.594901,9.417469e-08,0.000051,0.534709,0.000015,0.000976,0.490631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3081073,3,31,8,3,2017,9,B,6,POULTRY,True,...,6.339158,6.024116e+00,6.024116e+00,6.314312,5.992898e+00,5.992898,6.287649,5.974924,5.974924,6.252895
3081074,3,31,8,3,2017,9,B,6,PREPARED FOODS,False,...,3.955965,4.992324e+00,4.992324e+00,4.043447,4.949581e+00,4.949581,4.171029,4.907080,4.907080,4.302869
3081075,3,31,8,3,2017,9,B,6,PRODUCE,True,...,7.793466,7.704375e+00,7.704375e+00,7.737799,7.638017e+00,7.638017,7.681655,7.568340,7.568340,7.619194
3081076,3,31,8,3,2017,9,B,6,SCHOOL AND OFFICE SUPPLIES,True,...,5.281222,4.901019e+00,4.901019e+00,5.157989,4.962162e+00,4.962162,5.008067,5.005573,5.005573,4.814117


In [19]:
y_train

0             2.0
1             0.0
2             2.0
3          1091.0
4             0.0
            ...  
2442048      25.0
2442049      15.0
2442050       0.0
2442051     182.0
2442052       0.0
Name: sales, Length: 2442053, dtype: float64

In [20]:
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x)) 
X_val = X_val.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x)) 
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x)) 

In [21]:
hyper_params = {'task': 'train','boosting_type': 'gbdt','objective': 'regression','metric': ['l1','l2'],'learning_rate': 0.1,
'feature_fraction': 0.9,'bagging_fraction': 0.7,'bagging_freq': 10,'verbose': 0,"max_depth": 50,"num_leaves": 128,"max_bin": 512}

gbm = lgb.LGBMRegressor(**hyper_params)

gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='l1')

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_fr

LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, feature_fraction=0.9,
              max_bin=512, max_depth=50, metric=['l1', 'l2'], num_leaves=128,
              objective='regression', task='train', verbose=0)

In [28]:
y_pred = gbm.predict(X_val)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10


In [30]:
y_pred_0 = y_pred.clip(min=0)

In [31]:
root_mean_squared_log_error(y_val, y_pred_0)

0.6175265283399681

In [23]:
results = pd.concat([y_val.reset_index(drop=True), pd.Series(y_pred)], axis=1).rename(columns={'sales': 'y_val', 0: 'y_pred'})

results['y_pred'] = results['y_pred'].clip(0)

results = results[results['y_val'] > 10]

results

,y_val,y_pred
3,19.000,106.410467
5,119.630,121.568427
6,181.000,255.618494
9,165.181,191.744155
10,95.198,79.172085
...,...,...
610509,438.133,343.113844
610510,154.553,119.901675
610511,2419.729,2109.172870
610512,121.000,170.938761


In [24]:
y_pred = gbm.predict(X_test)
y_pred

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10


array([3.05620206e+00, 1.44531563e+00, 1.33109821e+01, ...,
       1.98464858e+03, 1.47236054e+02, 1.80003274e+01])

In [25]:
data_test['id']
output = pd.DataFrame(index=data_test['id'])
output['sales'] = y_pred
output['sales'] = output['sales'].clip(0)

In [26]:
output.to_csv('data/submission.csv')